# (WIP) Analysis of Shortcut Data

_NB: If you're new to Shortcut or this project, check out the [GettingStarted.ipynb](GettingStarted.ipynb) notebook first._

If you're looking to do analysis and reporting of data using Shortcut's API, this notebook will walk you through some example analyses using Pandas, NumPy, DuckDB, and matplotlib.

In [28]:
from scapi import ShortcutClient
client = ShortcutClient().validate()

In [31]:
import duckdb
import pandas as pd

## Velocity (Stories)

Velocity in project management is a measure of throughput: how much work did we complete within a given context?

A "given context" is most often a time interval. Shortcut's Iterations provide a shared view on such time intervals, so we'll use that to calculate how many stories get completed per iteration.

We can either request all iterations, or we can request all stories completed within a certain time frame and then group by their iterations.
While the former is simpler, the latter is more future-proof, in that Shortcut's v3 `/iterations` endpoint is not paginated.

In [72]:
# Simple (but less performant) approach for all iterations
its = client.get_json('/iterations')
df_its = pd.DataFrame(its)
len(its)

48

In [86]:
# Get completed stories, then fetch just their iterations
completed_stories = client.post_json('/stories/search',
                                     {
                                         'archived': False,
                                         'completed_at_start': '2023-10-01T00:00:00Z',
                                         'completed_at_end':   '2024-10-01T00:00:00Z',
                                     })
it_ids = {s['iteration_id'] for s in completed_stories if s['iteration_id'] is not None}
its = [client.get_json(f"/iterations/{id}") for id in it_ids]
done_its = [it for it in its if it['status'] == 'done']
df_its = pd.DataFrame(done_its)
df_its.head()

,app_url,description,entity_type,labels,mention_ids,member_mention_ids,associated_groups,name,global_id,label_ids,updated_at,group_mention_ids,end_date,follower_ids,group_ids,start_date,status,id,stats,created_at
0,https://app.shortcut.com/shortcut-pivotal-impo...,,iteration,[],[],[],[{'group_id': '66105996-75ed-46b7-9162-319ba97...,PT 240,v2:it:66105996-53b7-45f5-8494-6303221f1296:28169,[],2024-04-24T14:44:49Z,[],2024-04-14,[],[],2024-04-08,done,28169,"{'num_points_done': 139, 'num_related_document...",2024-04-24T14:44:49Z
1,https://app.shortcut.com/shortcut-pivotal-impo...,,iteration,[],[],[],[{'group_id': '66105996-75ed-46b7-9162-319ba97...,PT 238,v2:it:66105996-53b7-45f5-8494-6303221f1296:28170,[],2024-04-24T14:44:49Z,[],2024-03-31,[],[],2024-03-25,done,28170,"{'num_points_done': 1, 'num_related_documents'...",2024-04-24T14:44:49Z
2,https://app.shortcut.com/shortcut-pivotal-impo...,,iteration,[],[],[],[{'group_id': '66105996-75ed-46b7-9162-319ba97...,PT 240,v2:it:66105996-53b7-45f5-8494-6303221f1296:40464,[],2024-10-31T16:27:32Z,[],2024-04-14,[],[],2024-04-08,done,40464,"{'num_points_done': 102, 'num_related_document...",2024-10-31T16:27:32Z
3,https://app.shortcut.com/shortcut-pivotal-impo...,,iteration,[],[],[],[{'group_id': '66105996-75ed-46b7-9162-319ba97...,PT 238,v2:it:66105996-53b7-45f5-8494-6303221f1296:40466,[],2024-10-31T16:27:33Z,[],2024-03-31,[],[],2024-03-25,done,40466,"{'num_points_done': 1, 'num_related_documents'...",2024-10-31T16:27:33Z
4,https://app.shortcut.com/shortcut-pivotal-impo...,,iteration,[],[],[],[{'group_id': '66105996-75ed-46b7-9162-319ba97...,PT 243,v2:it:66105996-53b7-45f5-8494-6303221f1296:40467,[],2024-10-31T16:27:33Z,[],2024-05-05,[],[],2024-04-29,done,40467,"{'num_points_done': 37, 'num_related_documents...",2024-10-31T16:27:33Z


If our analysis requires all of the stories in each iteration, we can request those easily:

In [89]:
stories = [
    s
    for it_id in df_its['id']
    for s in client.get_json(f"/iterations/{it_id}/stories")
]
df_stories = pd.DataFrame(stories)
len(df_stories)

2353

In [46]:
df_stories.head()

,app_url,archived,started,story_links,entity_type,labels,task_ids,mention_ids,member_mention_ids,story_type,...,blocked,project_id,linked_file_ids,deadline,stats,comment_ids,created_at,moved_at,lead_time,cycle_time
0,https://app.shortcut.com/shortcut-pivotal-impo...,False,False,[],story,[],[],[],[],bug,...,False,None,[],None,{'num_related_documents': 0},[],2024-04-05T20:05:42Z,2024-04-05T20:05:42Z,NaN,NaN
1,https://app.shortcut.com/shortcut-pivotal-impo...,False,False,"[{'entity_type': 'story-link', 'object_id': 43...",story,[],[],[],[],feature,...,False,None,[],None,{'num_related_documents': 0},[],2024-11-22T16:05:22Z,2024-11-22T16:05:22Z,NaN,NaN
2,https://app.shortcut.com/shortcut-pivotal-impo...,False,True,"[{'entity_type': 'story-link', 'object_id': 43...",story,[],[31],[],[],chore,...,False,None,[],2024-04-12T20:05:42Z,{'num_related_documents': 0},[34],2024-04-05T20:05:42Z,2024-04-05T20:05:42Z,NaN,NaN
3,https://app.shortcut.com/shortcut-pivotal-impo...,False,True,[],story,[{'app_url': 'https://app.shortcut.com/shortcu...,[],[],[],feature,...,False,None,[],None,{'num_related_documents': 0},[],2019-09-13T00:00:00Z,2024-04-24T14:44:51Z,0.0,0.0
4,https://app.shortcut.com/shortcut-pivotal-impo...,False,False,[],story,[{'app_url': 'https://app.shortcut.com/shortcu...,[],[],[],chore,...,False,None,[],2024-03-29T00:00:00Z,{'num_related_documents': 0},[],2024-03-25T00:00:00Z,2024-04-24T14:44:55Z,NaN,NaN


Last five iterations, leveraging DuckDB's seamless integration with Pandas:

In [90]:
df_recent_its = duckdb.sql("""
SELECT it.id, it.name, it.start_date FROM df_its it
ORDER BY it.start_date DESC
LIMIT 5;
""")
df_recent_its

┌───────┬─────────┬────────────┐
│  id   │  name   │ start_date │
│ int64 │ varchar │  varchar   │
├───────┼─────────┼────────────┤
│ 40467 │ PT 243  │ 2024-04-29 │
│ 41773 │ PT 243  │ 2024-04-29 │
│ 33597 │ PT 243  │ 2024-04-29 │
│ 30791 │ PT 243  │ 2024-04-29 │
│ 34909 │ PT 243  │ 2024-04-29 │
└───────┴─────────┴────────────┘

In [112]:
# TODO This isn't a DataFrame, it's a DuckDB relation.
df_velocity = duckdb.sql("""
SELECT iteration_id, count(*) AS num_completed_stories
  FROM df_stories
 WHERE iteration_id IN (SELECT id FROM df_recent_its)
GROUP BY iteration_id
ORDER BY iteration_id DESC;
""").to_df()
df_velocity

,iteration_id,num_completed_stories
0,41773,226
1,40467,226
2,34909,226
3,33597,226
4,30791,3


In [106]:
df_velocity['num_completed_stories'].describe()

count      5.000000
mean     181.400000
std       99.728632
min        3.000000
25%      226.000000
50%      226.000000
75%      226.000000
max      226.000000
Name: num_completed_stories, dtype: float64

In [110]:
from functools import partial
qs = [0.25, 0.75, 0.95]
(q_25, q_75, q_95) = [partial(pd.Series.quantile, q=q) for q in qs]
q_25.__name__ = "25%"
q_75.__name__ = "75%"
q_95.__name__ = "95%"
    
df_velocity['num_completed_stories'].agg(["count", "mean", "std", "min", q_25, "median", q_75, q_95, "max"])

count       5.000000
mean      181.400000
std        99.728632
min         3.000000
25%       226.000000
median    226.000000
75%       226.000000
95%       226.000000
max       226.000000
Name: num_completed_stories, dtype: float64

## TODO

* Plotting